# Get members' nutrition info 
- get member's average calories, sodium, protein, fat, carbs (per serving per recipe) 

In [1]:
import pymongo 
import boto3 
from bs4 import BeautifulSoup as BS
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import community
import matplotlib.pyplot as plt
import networkx as nx

In [2]:
client = pymongo.MongoClient('mongodb://localhost:27017/')
allrecipes_db = client.allrecipes
members_coll = allrecipes_db.members
recipes_coll = allrecipes_db.recipes

In [14]:
members_coll.find_one().keys()

[u'followers_dict',
 u'reviews_dict',
 u'following_id_list',
 u'madeits_recipe_id_list',
 u'favorites_dict',
 u'reviews_recipe_id_list',
 u'nutrition_avg_vals',
 u'following_dict',
 u'member_ID',
 u'favorites_recipe_id_list',
 u'_id',
 u'aboutme',
 u'madeits_dict',
 u'followers_id_list']

In [22]:
eg_mem = members_coll.find_one({'madeits_recipe_id_list': {'$exists':True, '$size':20}})

In [24]:
eg_mem['member_ID']

u'1775895'

In [57]:
recipes = set(eg_mem['madeits_dict'].values() + eg_mem['favorites_dict'].values())

In [59]:
recipes

{u'102133',
 u'13482',
 u'14325',
 u'14397',
 u'146238',
 u'16359',
 u'16611',
 u'18074',
 u'19304',
 u'220325',
 u'220751',
 u'221091',
 u'221988',
 u'222615',
 u'222932',
 u'229780',
 u'22987',
 u'231030',
 u'232414',
 u'234415',
 u'238916',
 u'241267',
 u'25155',
 u'25364',
 u'38021',
 u'61674',
 u'62461962',
 u'62549419',
 u'63652701',
 u'64201826',
 u'64244607',
 u'64344329',
 u'64468946',
 u'75861',
 u'80658',
 u'84259',
 u'84270',
 u'8636',
 u'8778'}

In [64]:
import pandas as pd
nutes = {}
for r in recipes: 
    doc = recipes_coll.find_one({'recipe_ID': r})

    if not doc or not doc.get('Carbs'): 
        print "{} has no/incomplete document".format(r)
    else: 
        print doc['name'], doc.get('recipe_ID')
        nutes[doc['recipe_ID']]= [doc['name'][0],doc['Carbs'][0], doc['Calories'][0], doc['Protein'][0], doc['Fat'][0], doc['Sodium'][0]]
    npd = pd.DataFrame(index=nutes.keys(), data=nutes.values(), columns =['Name', 'Carbs', 'Calories', 'Protein', 'Fat', 'Sodium'] )

[u'Red Skinned Potato Salad'] 25155
[u"Kate's Black-Eyed Pea Salad"] 80658
64344329 has no/incomplete document
[u'Pasta Salad on a Stick'] 238916
[u'Marinated Flank Steak'] 18074
[u'Breakfast Sausage'] 16359
[u'Creamy Italian Dressing I'] 14325
[u'Lemony Cucumbers'] 14397
62461962 has no/incomplete document
[u'Slow-Cooker Corned Beef and Cabbage  '] 84270
[u'Winter Fruit Salad with Lemon Poppyseed Dressing'] 13482
[u'Cajun Chicken Pasta'] 8778
64201826 has no/incomplete document
[u'Margarita Cupcakes'] 241267
[u'Parmesan Chicken II'] 8636
[u'Country Apple Dumplings'] 102133
231030 has no/incomplete document
[u'Cornell Chicken'] 221091
[u'Thornehedge Peach Slaw'] 38021
[u'Baked Denver Omelet'] 229780
[u"Aimee's Quick Chicken"] 25364
[u'Chicken in a Pot'] 16611
[u'Oven BBQ Chicken Drumsticks'] 222932
64244607 has no/incomplete document
[u'Quick Chicken Piccata'] 220751
[u'Making Crispy Onion Rings'] 220325
[u'Red, White and Blue Strawberry Shortcake'] 22987
[u'Eggs Benedict Casserole'] 2

In [65]:
npd

,Name,Carbs,Calories,Protein,Fat,Sodium
25155,Red Skinned Potato Salad,16.2,430.0,9.5,36.9,536.0
14325,Creamy Italian Dressing I,2.4,139.0,1.0,14.6,153.0
18074,Marinated Flank Steak,3.4,275.0,14.8,22.5,935.0
16359,Breakfast Sausage,2.7,409.0,1.0,32.2,861.0
14397,Lemony Cucumbers,1.0,27.0,0.5,0.2,252.0
13482,Winter Fruit Salad with Lemon Poppyseed Dressing,21.0,277.0,4.9,20.6,201.0
8778,Cajun Chicken Pasta,54.0,935.0,43.7,61.7,1189.0
241267,Margarita Cupcakes,16.0,136.0,1.7,6.7,159.0
8636,Parmesan Chicken II,1.4,466.0,1.0,26.7,981.0
232414,The Girls' Chicken Sandwiches,51.0,437.0,21.9,10.1,379.0


In [66]:
npd.describe()

,Carbs,Calories,Protein,Fat,Sodium
count,31.000000,31.000000,31.000000,31.000000,31.000000
mean,20.767742,355.225806,16.151613,21.174194,806.225806
std,16.931064,204.489561,13.436837,15.421067,766.519307
min,1.000000,27.000000,0.500000,0.200000,1.000000
25%,5.550000,194.000000,2.200000,7.000000,238.000000
50%,17.500000,321.000000,17.500000,19.000000,712.000000
75%,33.000000,454.500000,24.600000,29.500000,1017.000000
max,54.000000,935.000000,43.700000,61.700000,3588.000000
